In [1]:
from neo4j import GraphDatabase
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier

In [2]:
# Neo4j driver
driver = GraphDatabase.driver('neo4j://localhost:7687', auth=('neo4j', 'letmein'))

## Agenda
In this example, you will reproduce the protein role classification task from the original GraphSAGE article. The task is to classify protein roles in terms of their cellular function across various protein-protein interaction graphs (PPI). The dataset contains 22 PPI graphs, with each graph corresponding to a different human tissue. The average PPI graph contains 2373 nodes, with an average degree of 28.8. There are available predefined positional gene sets, motif gene sets, and immunological signatures for each protein in the network. Based on those features and their connections, you will predict the roles of proteins in the network. You will train both the classification and GraphSAGE model on 20 graphs and then average prediction F1 scores on two test graphs.
## Graph model
As mentioned, we are dealing with a protein-protein interaction network. This is a monopartite network, where nodes represent proteins and relationships represent their interactions.

Additionally, the protein nodes have the predefined features stored as a property. The embeddings_all property contains all 50 features stored as a list of floats. I have also prepared the decoupled properties, where the embedding_x property holds a single feature and x ranges from 0 to 49. You will see later in the blog post why the decoupled properties are required. The protein nodes also contain a secondary label that could be either Train or Test. With the help of the secondary label, you can easily perform the train-test data split.

## Classification using predefined features
To get a baseline f1 score, you will first train the classification model using only the predefined features available for proteins. The code is identical to the code found in the official GraphSAGE repository, where they used the Stochastic Gradient Descent classifier model to train and predict protein roles. The only difference is that here you will be fetching the data from a Neo4j database instance.

In [3]:
raw_train_data_query = """
MATCH (t:Train)
RETURN t.class as class, t.embeddings_all as features
"""

raw_test_data_query = """
MATCH (t:Test)
RETURN t.class as class, t.embeddings_all as features
"""

with driver.session() as session:
    # Fetch training data
    train_results = session.run(raw_train_data_query)
    train_results_df = pd.DataFrame([dict(r) for r in train_results])
    
    # Fetch test data
    test_results = session.run(raw_test_data_query)
    test_results_df = pd.DataFrame([dict(r) for r in test_results])

In [ ]:
log = MultiOutputClassifier(SGDClassifier(loss="log"), n_jobs=10)
log.fit(train_results_df['features'].to_list(), train_results_df['class'].to_list())

print(f1_score(test_results_df['class'].to_list(), 
               log.predict(test_results_df['features'].to_list()), average="micro"))

Before you can execute any graph algorithms, you have to project the in-memory graph via the Graph Loader component. You can use either native projection or cypher projection to load the in-memory graph. 
In this example, you will use the native projection feature to load the in-memory graph. To start, you will project the training data and store it as a named graph in the Graph Catalog. The current implementation of the GraphSAGE algorithm supports only node features that are of type Float. For this reason, you will include the decoupled node properties ranging from embedding_0 to embedding_49 in the graph projection instead of a single property embeddings_all, which holds all the node features in the form of a list of Floats. Additionally, you will treat the projected graph as undirected.

In [5]:
with driver.session() as session:
    session.run("""UNWIND range(0,49) as i
                   WITH collect('embedding_' + toString(i)) as embeddings
                   CALL gds.graph.create('train','Train',
                    {INTERACTS:{orientation:'UNDIRECTED'}}, {nodeProperties:embeddings}) 
                   YIELD graphName, nodeCount, relationshipCount
                   RETURN graphName, nodeCount, relationshipCount""")

Next, you will train the GraphSAGE model. The model's hyper-parameter settings were mostly copied from the original paper. I have noticed that the lower learning-rate setting had the most impact on the downstream classification accuracy. Another import hyper-parameter is the samplingSizes parameter, where the size of the list determines the number of layers (defined as K parameter in the paper), and the values determine how many nodes will be sampled by the layers.

In [6]:
with driver.session() as session:
    session.run("""
        UNWIND range(0,49) as i
        WITH collect('embedding_' + toString(i)) as embeddings
        CALL gds.beta.graphSage.train('train',{
          modelName:'proteinModel',
          aggregator:'pool',
          batchSize:512,
          activationFunction:'relu',
          epochs:10,
          sampleSizes:[25,10],
          learningRate:0.0000001,
          embeddingDimension:256,
          featureProperties:embeddings})
        YIELD modelInfo
        RETURN modelInfo""")

The training process took around 20 minutes on my laptop. After the training process finishes, the GraphSAGE model will be stored in the model catalog. You can now use this model to induce node embeddings on any projected graph with the same node properties used during the training. Before testing the downstream classification accuracy, you have to load the test data as an in-memory graph in the Graph Catalog.

In [7]:
with driver.session() as session:
    session.run("""
        UNWIND range(0,49) as i
        WITH collect('embedding_' + toString(i)) as embeddings
        CALL gds.graph.create('test','Test',{INTERACTS:{orientation:'UNDIRECTED'}}, 
          {nodeProperties:embeddings}) 
        YIELD graphName, nodeCount, relationshipCount
        RETURN graphName, nodeCount, relationshipCount""")

With the GraphSAGE model trained and both the training and test data projected as an in-memory graph, you can go ahead and calculate the f1 score using the GraphSAGE embeddings in a downstream classification model. Remember, the GraphSAGE model has not observed the test data during the training phase.

In [ ]:
graphsage_train_data = """
CALL gds.beta.graphSage.stream('train', {modelName:'proteinModel'})
YIELD nodeId, embedding
RETURN gds.util.asNode(nodeId).class as class, embedding as features
"""

graphsage_test_data = """
CALL gds.beta.graphSage.stream('test', {modelName:'proteinModel'})
YIELD nodeId, embedding
RETURN gds.util.asNode(nodeId).class as class, embedding as features
"""
with driver.session() as session:
    # Fetch training data
    train_results = session.run(graphsage_train_data)
    train_results_df = pd.DataFrame([dict(t) for t in train_results])
    # Fetch test data
    test_results = session.run(graphsage_test_data)
    test_results_df = pd.DataFrame([dict(t) for t in test_results])

# Train the SGD classifier
log = MultiOutputClassifier(SGDClassifier(loss="log"), n_jobs=10)
log.fit(train_results_df['features'].to_list(), train_results_df['class'].to_list())

# Calculate the f1 score on test data
print(f1_score(test_results_df['class'].to_list(), 
               log.predict(test_results_df['features'].to_list()), average="micro"))